In [ ]:
# !pip install autofeat

In [ ]:
# !pip install sweetviz

In [ ]:
# !pip install GML

In [ ]:
# !pip install optuna

In [ ]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.neural_network import MLPRegressor
from sklearn.cluster import KMeans
from sklearn.ensemble import ExtraTreesRegressor, VotingRegressor, HistGradientBoostingRegressor
from sklearn.linear_model import BayesianRidge, LinearRegression, LassoLars, Ridge, ElasticNet

# import sweetviz as sv

# import smogn

import matplotlib.pyplot as plt

import seaborn as sns

from xgboost import XGBRegressor

from catboost import CatBoostRegressor

from lightgbm import LGBMRegressor

import tqdm

from category_encoders import TargetEncoder

# import optuna

from scipy import stats
from scipy.stats import norm, skew

# from autofeat import AutoFeatRegressor

# from pystacknet.pystacknet import StackNetRegressor

from GML.Ghalat_Machine_Learning import Ghalat_Machine_Learning

In [ ]:
def accuracy(y_true,y_pred):
  return 100 * max(0,1-np.sqrt(mean_squared_error(y_true,y_pred)))

In [ ]:
path = '/kaggle/input/he-mlcomp/'

In [ ]:
train = pd.read_csv(path+'Train.csv')
test = pd.read_csv(path+'Test.csv')

In [ ]:
train.columns

In [ ]:
train.head()

In [ ]:
types = {
    'type0':'M2',
    'type1':'M2',
    'type2':'L2',
    'type4':'L2',
    'type3':'L2'
}

In [ ]:
train['COMPType'] = train['Compensation_and_Benefits'].map(types)
test['COMPType'] = test['Compensation_and_Benefits'].map(types)

In [ ]:
test_eid = test['Employee_ID'].copy()

In [ ]:
pvar = 'Hometown'

In [ ]:
le = LabelEncoder()
train[pvar] = le.fit_transform(train[pvar])
test[pvar] = le.fit_transform(test[pvar])

In [ ]:
train['Employee_ID'] = train['Employee_ID'].apply(lambda x: int(x.split('_')[1]))
test['Employee_ID'] = test['Employee_ID'].apply(lambda x: int(x.split('_')[1]))

In [ ]:
"""
cols = train.columns[train.dtypes == 'object']
for col in cols:
    train[col].interpolate(method='linear',inplace=True)
    test[col].interpolate(method='linear',inplace=True)
"""
train.fillna(-999,inplace=True)
test.fillna(-999,inplace=True)

In [ ]:
test.sort_values('Age',inplace=True)
train.sort_values('Age',inplace=True)

In [ ]:
grid_lgbm = {
    'max_depth':16,
    'learning_rate':0.002,
    'n_estimators':1704,
    'min_child_weight':17,
    'eta':0.364
}

In [ ]:
g_var = 'Unit'

In [ ]:
minn = dict(train.groupby(g_var)['Attrition_rate'].min())
mediann = dict(train.groupby(g_var)['Attrition_rate'].mean())
maxx = dict(train.groupby(g_var)['Attrition_rate'].max())
stdd = dict(train.groupby(g_var)['Attrition_rate'].std())

train['_Min'] = train[g_var].map(minn)
train['_Avg'] = train[g_var].map(mediann)
train['_Max'] = train[g_var].map(maxx)
train['_Std'] = train[g_var].map(stdd)
test['_Min'] = test[g_var].map(minn)
test['_Avg'] = test[g_var].map(mediann)
test['_Max'] = test[g_var].map(maxx)
test['_Std'] = test[g_var].map(stdd)

In [ ]:
cols = train.columns[train.dtypes == 'object']
"""
enc = TargetEncoder()
train[cols] = enc.fit_transform(train[cols],train['Attrition_rate'])
test[cols] = enc.transform(test[cols])
"""
for col in cols:
  dummies = pd.get_dummies(train[col])
  train.drop([col],axis=1,inplace=True)
  train = pd.concat([train,dummies],axis=1)
  dummies = pd.get_dummies(test[col])
  test.drop([col],axis=1,inplace=True)
  test = pd.concat([test,dummies],axis=1)

"""
for col in test.columns:
    if train[col].isnull().any():
        model = LGBMRegressor(**grid_lgbm)
        t1 = train[~pd.isna(train[col])].copy()
        t1.drop(['Attrition_rate'],axis=1,inplace=True)
        t2 = train[pd.isna(train[col])].copy()
        t2.drop(['Attrition_rate','Age'],axis=1,inplace=True)
        t3 = test[pd.isna(test[col])].copy()
        t3.drop(['Age'],axis=1,inplace=True)
        model.fit(t1.drop([col],axis=1),t1[col])
        train.loc[t2.index,col] = model.predict(t2)
        test.loc[t3.index,col] = model.predict(t3)
"""
"""
for cols in test.columns:
    train[cols].fillna(train[cols].mean(),inplace=True)
    test[cols].fillna(test[cols].mean(),inplace=True)
"""
pass

In [ ]:
train['Can_Work_More_Years'] = 65 - train['Age'] - train['Time_of_service']
test['Can_Work_More_Years'] = 65 - test['Age'] - test['Time_of_service']

In [ ]:
"""
funcs = [np.multiply, np.add, np.subtract, np.multiply, np.divide]

for i, func in enumerate(funcs):
    train['Age_TOS'+str(i+1)] = func(train['Age'],train['Time_of_service'])
    test['Age_TOS'+str(i+1)] = func(test['Age'],test['Time_of_service'])

    train['AT_GR'+str(i+1)] = func(train['Age_TOS'+str(i+1)], train['growth_rate'])
    test['AT_GR'+str(i+1)] = func(test['Age_TOS'+str(i+1)], test['growth_rate'])

    train['TS_GR'+str(i+1)] = func(train['Time_of_service'], train['growth_rate'])
    test['TS_GR'+str(i+1)] = func(test['Time_of_service'], test['growth_rate'])

    train['PS_WLB'+str(i+1)] = func(train['Pay_Scale'], train['Work_Life_balance'])
    test['PS_WLB'+str(i+1)] = func(test['Pay_Scale'], test['Work_Life_balance'])

    train['Age_TSP'+str(i+1)] = func(train['Age'], train['Time_since_promotion'])
    test['Age_TSP'+str(i+1)] = func(test['Age'],test['Time_since_promotion'])

    train['Age_EMP'+str(i+1)] = func(train['Age'], train['Employee_ID'])
    test['Age_EMP'+str(i+1)] = func(test['Age'], test['Employee_ID'])

    train['TOS_EMP'+str(i+1)] = func(train['Time_of_service'],train['Employee_ID'])
    test['TOS_EMP'+str(i+1)] = func(test['Time_of_service'],test['Employee_ID'])

    train['ATOS_EMP'+str(i+1)] = func(train['Age_TOS'+str(i+1)], train['Employee_ID'])
    test['ATOS_EMP'+str(i+1)] = func(test['Age_TOS'+str(i+1)], test['Employee_ID'])
    break
"""
var_names = ['VAR'+str(i) for i in range(1,8)]
sums_tr = [] 
subs_tr = [] 
mul_tr = [] 
div_tr = []
sums_ts = []
subs_ts = []
mul_ts = []
div_ts = []
for row in train[var_names].itertuples():
    sums_tr.append(row[1]+row[2]+row[3]+row[4]+row[5]+row[6]+row[7])
    subs_tr.append(row[1]-row[2]-row[3]-row[4]-row[5]-row[6]-row[7])
    mul_tr.append(row[1]*row[2]*row[3]*row[4]*row[5]*row[6]*row[7])
    div_tr.append(row[1]/row[2]/row[3]/row[4]/row[5]/row[6]/row[7])
    
for row2 in test[var_names].itertuples():
    sums_ts.append(row[1]+row[2]+row[3]+row[4]+row[5]+row[6]+row[7])
    subs_ts.append(row[1]-row[2]-row[3]-row[4]-row[5]-row[6]-row[7])
    mul_ts.append(row[1]*row[2]*row[3]*row[4]*row[5]*row[6]*row[7])
    div_ts.append(row[1]/row[2]/row[3]/row[4]/row[5]/row[6]/row[7])

train['sum_VARS'] = sums_tr
train['mul_VARS'] = mul_tr
train['div_VARS'] = div_tr
test['sum_VARS'] = sums_ts
test['mul_VARS'] = mul_ts
test['div_VARS'] = div_ts

train['mulvars_gr'] = train['growth_rate'] * train['mul_VARS']
test['mulvars_gr'] = test['growth_rate'] * test['mul_VARS']

In [ ]:
grid_bayesian = {
    'n_iter':10000,
    'tol':6.42e-06,
    'alpha_1':0.0001,
    'alpha_2':6.58e-05,
    'lambda_1':0.999,
    'lambda_2':2.24e-05,
    'compute_score':True,
    'fit_intercept':True,
    'normalize':False
}

grid_ridge = {
    'max_iter':90000,
    'alpha': 0.23143352151759083, 
    'fit_intercept': True, 
    'normalize': False, 
    'tol': 0.0005243805101392493, 
    'solver': 'saga'
}

In [ ]:
train = train.replace([np.inf, -np.inf], np.nan)
test = test.replace([np.inf, -np.inf], np.nan)

for col in test.columns:
    train[col].replace([np.inf,-np.inf],np.nan,inplace=True)
    test[col].replace([np.inf,-np.inf],np.nan,inplace=True)
    train[col].interpolate(method='linear',inplace=True)
    test[col].interpolate(method='linear',inplace=True)

In [ ]:
X = train.drop(['Attrition_rate'],axis=1)
y = train['Attrition_rate'].copy().values

In [ ]:
X = X.replace([np.inf, -np.inf], np.nan)
test = test.replace([np.inf, -np.inf], np.nan)
for cols in X.columns:
    X[cols].fillna(X[cols].mean(),inplace=True)
    test[cols].fillna(X[cols].mean(),inplace=True)

In [ ]:
lgbm = LGBMRegressor(**grid_lgbm)
lgbm.fit(X, y)

fi = pd.DataFrame()
fi['A'] = X.columns.values
fi['B'] = lgbm.feature_importances_

In [ ]:
fi.sort_values('B',ascending = False)

In [ ]:
fi.describe()

In [ ]:
cols_to_d = fi[fi['B'] == 0].loc[:,'A']
X.drop(cols_to_d,axis=1,inplace=True)
test.drop(cols_to_d,axis=1,inplace=True)

In [ ]:
"""
def perc(trial):
    p1 = trial.suggest_uniform('p1',0.1,0.99)
    p2 = trial.suggest_uniform('p2',0.1,0.99)
    _scores = []
    for v in tqdm.tqdm(np.unique(train[pvar])):
        scores = []
        X_ = X[X[pvar]==v].copy()
        indexes = X[X[pvar]==v].index
        y_ = y[indexes]

        ridge = Ridge(**grid_ridge)
        br = BayesianRidge()
        for tr_in, val_in in KFold().split(X_, y_):
            br.fit(X_.iloc[tr_in], y_[tr_in])
            ridge.fit(X_.iloc[tr_in], y_[tr_in])
            preds = (br.predict(X_.iloc[val_in]) * p1 + ridge.predict(X_.iloc[val_in]) * p2)
            scores.append(accuracy(y_[val_in],preds))
        _scores.append(np.mean(scores))
    return -np.mean(_scores)
"""
pass

In [ ]:
# study = optuna.create_study()

In [ ]:
# study.optimize(perc)

In [ ]:
p1 = 0.9
p2 = 0.15

In [ ]:
ss = pd.DataFrame()
ss['Emp_ID'] = test['Employee_ID'].copy()
ss['Employee_ID'] = test_eid
ss['Attrition_rate'] = np.nan

In [ ]:
"""
_scores = []
for v in tqdm.tqdm(np.unique(train[pvar])):
    scores = []
    X_ = X[X[pvar]==v].copy()
    X_.drop(pvar,axis=1,inplace=True)
    indexes = X[X[pvar]==v].index
    y_ = y[indexes]
    sgd = LinearRegression()
    try:
        for tr_in, val_in in KFold().split(X_, y_):
            sgd.fit(X_.iloc[tr_in], y_[tr_in])
            preds = (sgd.predict(X_.iloc[val_in])*1.05)
            scores.append(accuracy(y_[val_in],preds))
        _scores.append(np.mean(scores))
    except:
        pass
    sgd.fit(X_,y_)
            
    test_ = test[test[pvar]==v].copy()
    test_.drop(pvar,axis=1,inplace=True)
    if test_.empty:
        continue
    e_ind = test[test[pvar]==v].loc[:,'Employee_ID']
    preds = (sgd.predict(test_)*1.05)
    ind = ss[ss['Emp_ID'].isin(e_ind)].index
    ss.at[ind,'Attrition_rate'] = preds
"""
pass

In [ ]:
"""
scores = []
for tr_in, val_in in KFold(shuffle=False).split(X, y):
    reg = StackingRegressor(estimators=estimators,final_estimator=BayesianRidge(**grid_bayesian))
    reg.fit(X.iloc[tr_in], y[tr_in])
    preds = reg.predict(X.iloc[val_in])
    sco = accuracy(y[val_in],preds)
    print(sco)
    scores.append(sco)
"""
pass

In [ ]:
_scores = []
for v in tqdm.tqdm(np.unique(train[pvar])):
    scores = []
    X_ = X[X[pvar]==v].copy()
    X_.drop(pvar,axis=1,inplace=True)
    indexes = X[X[pvar]==v].index
    y_ = y[indexes]
    y_ = np.log1p(y_)
    br = BayesianRidge()
    ridge = Ridge(**grid_ridge)
    try:
        for tr_in, val_in in KFold().split(X_, y_):
            br.fit(X_.iloc[tr_in], y_[tr_in])
            ridge.fit(X_.iloc[tr_in], y_[tr_in])
            preds = (br.predict(X_.iloc[val_in])*p1 + ridge.predict(X_.iloc[val_in])*p2)
            preds = np.expm1(preds)
            scores.append(accuracy(y_[val_in],preds))
        _scores.append(np.mean(scores))
    except:
        pass
    br.fit(X_,y_)
    ridge.fit(X_,y_)
    test_ = test[test[pvar]==v].copy()
    if test_.empty:
        continue
    test_.drop(pvar,axis=1,inplace=True)
    e_ind = test[test[pvar]==v].loc[:,'Employee_ID']
    preds = br.predict(test_)*p1 + ridge.predict(test_)*p2
    preds = np.expm1(preds)
    ind = ss[ss['Emp_ID'].isin(e_ind)].index
    ss.at[ind,'Attrition_rate'] = preds

In [ ]:
_scores

In [ ]:
np.mean(_scores)

In [ ]:
ss.drop(['Emp_ID'],axis=1,inplace=True)
ss.to_csv('submission.csv',index=False)

In [ ]:
train['Attrition_rate'].describe()

In [ ]:
ss['Attrition_rate'].describe()

In [ ]:
ss